In [1]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random 
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import string
from nltk.corpus import stopwords
plt.ion()

In [2]:
reviews = pd.read_csv("model/master_reviews.csv")

In [3]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,text_length,label
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,total bill horrible service 8gs crooks actuall...,2013-05-07 04:34:36,129,0
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5,0,0,0,adore travis hard rocks new kelly cardenas sal...,2017-01-14 21:30:33,1064,1
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5,3,0,0,say office really together organized friendly ...,2016-11-09 20:09:03,369,1
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,went lunch steak sandwich delicious caesar sal...,2018-01-09 20:56:38,279,1
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1,7,0,0,today second three sessions paid although firs...,2018-01-30 23:07:38,2021,0


In [4]:
reviews.shape

(6685899, 11)

In [5]:
reviews_mod = reviews.dropna()

In [6]:
reviews_mod.shape

(6685820, 11)

In [7]:
class SentimentAnalysis:
    def __init__(self):
        
        self.map_word = {}
        self.map_index = {}
        self.x_y_pairs = {}
        self.pairs_map = {}
        
#    def word_embed(self)
    
#     def word2vec(self):
#         self.window_size = 2
#         self.epochs = 100
#         self.embeddings = 10
#         self.vocab_size = length(sla.pairs_map[self.window_size].keys())
        
#         weight1 = torch.randn(self.embeddings,self.vocab_size)
#         weight2 = torch.randn(self.vocab_size,self.embeddings)
        
#         for epoch in epochs:
#             for center,context in 
    
    def NGRAM(self):
        pass

In [30]:
class TrainSelfEmbedding:
    
    def __init__(self):
        self.window_size = 2
        self.epochs = 100
        self.embeddings = 10
        self.epoch_loss = 0
        
    def train(self,pairs_map):
        
        self.vocab_size = len(sla.pairs_map[self.window_size])
        
        weight1 = torch.randn(self.embeddings,self.vocab_size).float()
        weight2 = torch.randn(self.vocab_size,self.embeddings).float()
        
        print(self.vocab_size)
        
        for center,target in pairs_map[self.window_size]:
            
            inpt = torch.zeros(self.vocab_size).float()
            inpt[center] = 1.0
            z1 = torch.matmul(weight1,inpt)
            z2 = torch.matmul(weight2,z1)
            
            log_output = torch.log_softmax(z2)
            
            loss = torch.nn.functional.nll_loss(log_output.view(1,-1),torch.tensor([context]))
            
            total_loss += loss
            
            
            

In [8]:
def tokenize_vocab(map_word,map_index,x):
    
    idx = len(map_word.keys())
    ans = []
    #print(type(x),x)
    for word in x.split():
        if word not in map_word:
            idx+=1
            map_word[word] = idx
            map_index[idx] = word
        ans.append(idx)
            
    return ans


def build_context_center(x,pairs_map,window = 2):
    
    if window not in pairs_map:
        pairs_map[window] = []
        
    #x = x.split()
    for i in range(len(x)):
        
        for j in range(1,window+1):
            
            if (i-j)>0:
                pairs_map[window].append([x[i],x[i-j]])
                
            if (i+j)<len(x):
                pairs_map[window].append([x[i],x[i+j]])
                
                

In [9]:
%%time
sla = SentimentAnalysis()
reviews_mod['tokenized'] = reviews_mod.text.apply(lambda x: tokenize_vocab(sla.map_word,sla.map_index,x))

CPU times: user 2min 13s, sys: 3.12 s, total: 2min 16s
Wall time: 2min 16s


In [10]:
reviews_mod.text_length.max()

4755

In [11]:
%%time
_ = reviews_mod.iloc[:,].tokenized.apply(lambda x:build_context_center(x,sla.pairs_map))

CPU times: user 19min 8s, sys: 1min 13s, total: 20min 21s
Wall time: 20min 19s


In [12]:
sla.pairs_map[2][:10]

[[1, 2],
 [1, 3],
 [2, 3],
 [2, 4],
 [3, 2],
 [3, 4],
 [3, 5],
 [4, 3],
 [4, 5],
 [4, 2]]

In [13]:
%%time
import pickle as pkl
with open("model/map.pkl",'wb') as file:
    pkl.dump(sla.pairs_map,file)

CPU times: user 14min 46s, sys: 1min 36s, total: 16min 22s
Wall time: 18min 53s


In [14]:
%%time
with open("model/map_word.pkl",'wb') as file:
    pkl.dump(sla.map_word,file)

CPU times: user 1.53 s, sys: 241 ms, total: 1.77 s
Wall time: 2.14 s


In [15]:
%%time

with open("model/map_index.pkl",'wb') as file:
    pkl.dump(sla.map_index,file)

CPU times: user 1.45 s, sys: 225 ms, total: 1.67 s
Wall time: 1.97 s


In [28]:
self_embedding = TrainSelfEmbedding()

In [29]:
%%time
self_embedding.train(sla.pairs_map)

1475186332


NameError: name 'vocab_size' is not defined